In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName('app').getOrCreate()
sc = spark.sparkContext
data = sc.parallelize([(1, 3), (2, 5), (1, 7), (2, 8), (3, 2)])
sums = data.reduceByKey(lambda a, b: a + b)
print("reducedByKey:", sums.collect())

#groupByKey
group = data.groupByKey()
print("groupByKey")
for k,v in group.collect():
    print(f"Key: {k}, Values: {list(v)}")
    
#aggregateByKey
zero_value = 0  
def seq_op(accumulator, value):
    return accumulator + value
def comb_op(accumulator1, accumulator2):
    return accumulator1 + accumulator2
sums = data.aggregateByKey(zero_value, seq_op, comb_op)
print("aggregateByKey:",sums.collect())

#combineBytKey
def create_combiner(value):
    return (value, 1)
def merge_value(accumulator, value):
    total, count = accumulator
    return (total + value, count + 1)
def merge_combiners(accumulator1, accumulator2):
    total1, count1 = accumulator1
    total2, count2 = accumulator2
    return (total1 + total2, count1 + count2)
averages = data.combineByKey(create_combiner, merge_value, merge_combiners)
result = averages.mapValues(lambda accumulator: accumulator[0] / accumulator[1])
print("combineByKey:", result.collect())

reducedByKey: [(1, 10), (2, 13), (3, 2)]
groupByKey
Key: 1, Values: [3, 7]
Key: 2, Values: [5, 8]
Key: 3, Values: [2]
aggregateByKey: [(1, 10), (2, 13), (3, 2)]
combineByKey: [(1, 5.0), (2, 6.5), (3, 2.0)]
